In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.base import TransformerMixin, BaseEstimator
!pip install category_encoders
import category_encoders as ce 
from sklearn.model_selection import GridSearchCV


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
def replace(group):
    mean, std = group.mean(), group.std()
    outliers = (group - mean).abs() > 1.5*std
    group[outliers] = group[~outliers].mean()
    return group


In [4]:
def is_outlier(s):
    lower_limit = s.mean() - (s.std() * 2.5)
    upper_limit = s.mean() + (s.std() * 2.5)
    return ~s.between(lower_limit, upper_limit)

In [5]:
df=pd.read_csv("/content/Yad2cars2.csv")

In [6]:
df=df.drop_duplicates()


In [7]:
df.head()

,רכב,שנה,יד,נפח מנוע בסמק,מחיר
0,"שברולט מאליבו\nLT אוט' 3.5 (221 כ""ס)",2010,2,"3,500","22,500 ₪"
1,"פיג'ו 207\nTrendy אוט' 1.6 (120 כ""ס)",2009,3,"1,598","9,100 ₪"
2,"מיצובישי ספייס סטאר\nInstyle אוט' 1.2 (80 כ""ס)",2015,2,"1,193","28,000 ₪"
3,אאודי TT\nDesign קופה אוט' 2.0 (197 כ''ס),2019,1,"1,984","220,000 ₪"
4,מרצדס S-Class\nS560E AMG Line הייבריד אוט' 3.0...,2019,1,"2,996","630,000 ₪"


In [8]:
df.shape

(1457, 5)

In [9]:
df=df.rename(columns={"מחיר": "Price", "נפח מנוע בסמק": "Engine volume in cmc","יד":"Number of owners","שנה":"Year","רכב":"car"})


In [10]:
df.car=df.car.str.replace("(","@")
df["ph"]=df.car.map(lambda x:"".join(re.findall("@\d+",x)))
df['haybrid']=np.where(df.car.str.contains("הייבריד"),1,0)
df['brand']=df.car.apply(lambda x:x.split("\n")[0])
df['brand2']=df.brand.map(lambda x:"".join(re.findall("\w[a-z-A-Z-0-9]+",x)))
#df['brand2']=df.brand.apply(lambda x: if ( x.startswith("\w[a-z-A-Z-0-9]+")) then (x.split(" ")[1]) else( x.str.split(" ")[0]))
df['brand3']=df.brand.apply(lambda x:x.split(" ")[1] if x.startswith("\w[a-z-A-Z-0-9]+")else x.split(" ")[0])
df['Model']=df.brand.apply(lambda x:" ".join(x.split(" ")[0] if x.startswith("\w[a-z-A-Z-0-9]+")else x.split(" ")[1:4]))
df['liter']=df.car.map(lambda x:"".join(re.findall(r"\d\.\d{1}",x)))
df=df.replace('-',np.NaN)
df.Price=df.Price.apply(lambda x:x.replace("₪",""))
df['Number of owners']=df['Number of owners'].apply(lambda x:x.replace("10+","10"))
df['liter']=np.where(df.liter=='',"0",df.liter)
df.Price=df.Price.str.strip().replace('לא צוין מחיר','0')#.replace(",","").astype('float')
df.Price=df.Price.apply(lambda x:x.replace(",","")).astype('float')
df.ph=df.ph.apply(lambda x: re.sub("[1-3][0-9]{3}","",x ).replace("@",""))
df.ph=df.ph.apply(lambda x:np.nan if x=="" else x )
df['years_bin']=pd.qcut(df.Year.astype('float'),5,labels=False)
#*******
df=df[~((df.liter=='2.02.0')|(df.liter=='2.52.5')|(df.liter=='020.00020.000')|(df.liter=='0'))]


In [11]:
df.head()


,car,Year,Number of owners,Engine volume in cmc,Price,ph,haybrid,brand,brand2,brand3,Model,liter,years_bin
0,"שברולט מאליבו\nLT אוט' 3.5 @221 כ""ס)",2010,2,"3,500",22500.0,221,0,שברולט מאליבו,,שברולט,מאליבו,3.5,0.0
1,"פיג'ו 207\nTrendy אוט' 1.6 @120 כ""ס)",2009,3,"1,598",9100.0,120,0,פיג'ו 207,207,פיג'ו,207,1.6,0.0
2,"מיצובישי ספייס סטאר\nInstyle אוט' 1.2 @80 כ""ס)",2015,2,"1,193",28000.0,80,0,מיצובישי ספייס סטאר,,מיצובישי,ספייס סטאר,1.2,2.0
3,אאודי TT\nDesign קופה אוט' 2.0 @197 כ''ס),2019,1,"1,984",220000.0,197,0,אאודי TT,TT,אאודי,TT,2.0,4.0
4,מרצדס S-Class\nS560E AMG Line הייבריד אוט' 3.0...,2019,1,"2,996",630000.0,367,1,מרצדס S-Class,S-Class,מרצדס,S-Class,3.0,4.0


In [12]:

#from google.colab import drive
#drive.mount('/content/drive')

In [13]:
#df=df.convert_dtypes()
#*********
#df=df[~df['years_bin'].isna()]
#df[~df.groupby(['brand3','liter','years_bin'])['Price'].apply(is_outlier)]
#********
#df3.Price=df.Price.astype('int64')
#df3.groupby(['liter','years_bin']).Price.transform(replace)


In [14]:

df_new=df[['years_bin','Number of owners','Price','haybrid','brand3','liter']]
df_new.head()

,years_bin,Number of owners,Price,haybrid,brand3,liter
0,0.0,2,22500.0,0,שברולט,3.5
1,0.0,3,9100.0,0,פיג'ו,1.6
2,2.0,2,28000.0,0,מיצובישי,1.2
3,4.0,1,220000.0,0,אאודי,2.0
4,4.0,1,630000.0,1,מרצדס,3.0


In [15]:
df_new.columns

Index(['years_bin', 'Number of owners', 'Price', 'haybrid', 'brand3', 'liter'], dtype='object')

In [16]:
df_new=df_new[df_new['Price']!=0]
#df_new=df_new.convert_dtypes()
df_new["Number of owners"]=df_new["Number of owners"].astype('int')
df_new["liter"]=df_new["liter"].astype('float')

In [17]:

#df_new.liter.
#df_new.years_bin[]


In [18]:
class TargetRatingTransformer  (TransformerMixin, BaseEstimator):  # TransformerMixin needed for fit_transform
    
    def __init__(self):
        super().__init__() 
        self.ce_target = ce.TargetEncoder(cols =['brand3','years_bin'])

    
        
    def fit (self ,X ,y):
       self.ce_target.fit(X,y)
       return self
    
    
    def transform(self, X ):
       ce_target_df=self.ce_target.transform(X) 
       
        
       return   ce_target_df

In [19]:
X=df_new.drop(['Price'],axis=1)
y=df_new['Price']
X_train, X_test, y_train, y_test = train_test_split(  X, y, test_size=0.33) 

In [20]:
my_target_encoder=TargetRatingTransformer()
df_train=my_target_encoder.fit_transform(X_train,y_train)
df_test=my_target_encoder.transform(X_test)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [28]:
y_train.shape

(841,)

In [31]:
regr = RandomForestRegressor()
parameters = {'max_depth':[2, 10],'min_samples_split':[2,10]}
random_regr = GridSearchCV(regr, parameters,cv=10)

In [32]:
random_regr.fit(df_train,y_train)
y_pred=random_regr.predict(df_test)

In [33]:
mean_squared_error(y_test, y_pred)

1700742735.0863059